In [2]:
########################################### STUDY 1 ###################################################

# Import libraries
import pandas as pd
import numpy as np
import time
import matplotlib as plt

# Read both datasets and concatenate. 

data1 = pd.read_csv("tripdata_2017-01.csv")
data2 = pd.read_csv("tripdata_2017-02.csv")
data = pd.concat([data1,data2])

print("#############################################################################")
print("Shape of both datasets joint")
print(data.shape)

start = time.time()
# Change the format of variable to datetime and extract the hour
data['tpep_pickup_datetime'] = data['tpep_pickup_datetime'].astype('datetime64[ns]') 
data['tpep_pickup_hour'] = data.tpep_pickup_datetime.dt.hour


# Convert the discrete variable to continuous variable
col         = 'tpep_pickup_hour'
conditions  = [ (data[col] >= 0) & (data[col] < 6), 
                (data[col]>= 6) & (data[col] < 12), 
               (data[col]>= 12) & (data[col] < 18),
               (data[col]>= 18) & (data[col] <= 24) 
  ]
choices     = [ 'N', 'M', 'A', 'E']
    
data["tpep_pickup_hour_timezone"] = np.select(conditions, choices, default=np.nan)

# Now there are only four categories which express the time zone

print("#############################################################################")
print("Categories of the new variable")
print(data['tpep_pickup_hour_timezone'].unique())

# Relative frequencies
print("#############################################################################")
print("Relative frequencies")
print(data['tpep_pickup_hour_timezone'].value_counts())
end = time.time()
print(end-start)

# Barlot of the new variable created
plot = data['tpep_pickup_hour_timezone'].value_counts().plot(kind='bar',
                                    figsize=(14,8),
                                    title="Barplot of 'tpep_pickup_hour_timezone'")

plot.set_ylabel('Frequencies')
plot.set_xlabel('Categories')

FileNotFoundError: [Errno 2] No such file or directory: 'tripdata_2017-01.csv'

# Average total price per trip excluding taxes.

We select the columns corresponding to the fare, tip, tolls and extra amount and the improvement surcharge, excluding the taxes and we add them together.

In [1]:
no_taxes = data['fare_amount'] + data['tip_amount'] + data['tolls_amount'] + data['extra'] + data['improvement_surcharge']

NameError: name 'data' is not defined

Now, we compute the mean of this quantity.

In [ ]:
meanprice=no_taxes.mean()
print (meanprice, 'No taxes mean')

Let's compare this mean value to the mean of the total amount given to the driver.

In [ ]:
total = data['total_amount']
total_mean = total.mean()
print (total_mean, 'Total amount mean')

We can see that there is a difference of close to 0.5$ between these amounts, meaning that the fraction of the total amount spent in taxes is close to 0.5$ for every trip. Let's check this fact: 

In [ ]:
print(data['mta_tax'].mean())

# 3 - Type of payment in terms of different pick up zones

We are going to try first to do an analysisis that uses serial computation and basic libraries from python to analyse the mean price of travels for the different pick-up zones and payment methods.

We see that we have four levels for the payment method:

 - Credit Card "1"
 - Cash "2"
 - No charge "3"
 - Dispute "4"

In [ ]:
fourth_analysis=data.loc[:,['DOLocationID','payment_type','total_amount']]
fourth_analysis.groupby(['DOLocationID','payment_type'])['total_amount'].mean().unstack().plot()

We can see that there must be an error on on or few observations as the mean price for one of the zones goes up to almost 20000 dollars, therefore we are now going to remove it visualice a more reasonable plot.

In [ ]:
plot_fourth_analysis=fourth_analysis.index[(fourth_analysis['total_amount']>1000) | (fourth_analysis['total_amount']<0)]
fourth_analysis=fourth_analysis.drop(plot_fourth_analysis,axis=0)
fourth_analysis.groupby(['DOLocationID','payment_type'])['total_amount'].mean().unstack().plot(ylim=[0,100],)

As we can see, in general, the payment method most used for the most expensive trips is isually by credit card. However there is also a lot of trips that are out of charge and that those trips with disputes to pay are for low amounts.

Now let's check how is the mean price distributed for each payment method.

In [ ]:
fourth_analysis.groupby(["payment_type"])["total_amount"].plot(kind='density',xlim=[0,50],xlabel="Price")

We can see that morarles the main payment price is concentrated over the same quantity, between 5 and 10 dollars. Nevertheless we can see that in general most payment method used is the cash and the mean value of trips is a little bit lower than the credit ones. 

Now that we have completed our analysis using a serial version of our problem we can check the system time requiered for it.